In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Sociodemographic Imputation/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1009. NC ENVRIOSCAN/P1009.3. Analyses/P1009.3.2. Sociodemographic Imputation/Output')
cur_date = "123022"

library(readxl)
library(tidyverse)
library(reshape2)
library(missForest)

# reading in files
acs_df = data.frame(read_excel("ACS_Data_121222.xlsx", sheet = 2))
# need to make some of the cols numeric
acs_df[,4:30] = apply(acs_df[,4:30], 2, as.numeric)
wildfire_hazard_df = data.frame(read_excel("Wildfire_Hazard_121222.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_df[, 4:30], 2, as.numeric):
“NAs introduced by coe

In [2]:
head(acs_df)
head(wildfire_hazard_df)

,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Adult,Poverty_Elderly,Poverty_Male,Poverty_Female,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,20.8,14.6,26.4,28.0,54.8,0,45.3,0.0,31.3,13.9
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,33.0,19.2,27.3,37.8,44.6,0,0.0,82.7,16.2,39.0
3,14000US37001020300,203.00,Alamance,77.8,40.7,27.1,72.9,18.9,45.2,28.9,⋯,17.3,22.0,18.0,28.3,21.0,NA,10.9,36.4,26.7,20.1
4,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,26.8,14.0,30.0,33.3,19.9,NA,69.0,100.0,67.2,26.6
5,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,12.2,7.1,12.3,14.8,18.9,0,0.0,53.1,25.7,10.0
6,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,22.3,19.2,17.7,31.0,27.8,100,14.2,19.5,14.7,25.0


,GEO_ID,Census_Tract,County,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,14000US37001020100,201.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
2,14000US37001020200,202.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
3,14000US37001020301,203.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%
4,14000US37001020302,203.02,Alamance,278233.79700000002,92.237034639041696,1,0-20%
5,14000US37001020400,204.00,Alamance,278233.79700000002,92.237034639041696,1,0-20%
6,14000US37001020501,205.01,Alamance,278233.79700000002,92.237034639041696,1,0-20%


In [3]:
# seeing if the census tracts are consistent each file
length(unique(acs_df$Census_Tract))
length(unique(wildfire_hazard_df$Census_Tract))

[1] 1410

[1] 1776

In [4]:
# they're not so what tracts aren't consistent?
`%notin%` <- Negate(`%in%`)
wildfire_hazard_df %>%
    filter(Census_Tract %notin% unique(acs_df$Census_Tract)) %>%
    select(c("GEO_ID", "Census_Tract", "County")) %>%
    head()

,GEO_ID,Census_Tract,County
,<chr>,<dbl>,<chr>
1,14000US37007920302,9203.02,Anson
2,14000US37007980000,9800.00,Anson
3,14000US37019020107,201.07,Brunswick
4,14000US37019020108,201.08,Brunswick
5,14000US37019020205,202.05,Brunswick
6,14000US37019020206,202.06,Brunswick


In [5]:
# lot's of missing data
dim(full_join(acs_df, wildfire_hazard_df))
dim(inner_join(acs_df, wildfire_hazard_df))

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 3137   34

Joining, by = c("GEO_ID", "Census_Tract", "County")


[1] 1730   34

In [6]:
acs_whp_df = inner_join(acs_df, wildfire_hazard_df)

# need to make some of the cols numeric
acs_whp_df[,30:33] = apply(acs_whp_df[,30:33], 2, as.numeric)
head(acs_whp_df)

Joining, by = c("GEO_ID", "Census_Tract", "County")
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(acs_whp_df[, 30:33], 2, as.numeric):
“NAs introduced by coercion”


,GEO_ID,Census_Tract,County,Sex_Ratio,Median_Age,Population_Under_18,Population_Over_18,Population_Over_65,Race_White,Race_Black,⋯,Poverty_Black,Poverty_Asian,Poverty_Other_Race,Poverty_Mixed_Race,Poverty_Hispanic_Latino,Poverty_White,Avg_ACRES,Wildfire_Hazard_Potential_Mean,Wildfire_Hazard_Potential_Quintile_Number,Wildfire_Hazard_Potential_Quintile_Percentage
,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,14000US37001020100,201.00,Alamance,87.2,37.3,21.9,78.1,15.1,61.0,28.5,⋯,54.8,0,45.3,0.0,31.3,13.9,278233.8,92.23703,1,0-20%
2,14000US37001020200,202.00,Alamance,78.3,31.0,29.1,70.9,11.2,51.9,29.2,⋯,44.6,0,0.0,82.7,16.2,39.0,278233.8,92.23703,1,0-20%
3,14000US37001020400,204.00,Alamance,78.1,30.5,31.4,68.6,10.1,19.6,59.0,⋯,19.9,NA,69.0,100.0,67.2,26.6,278233.8,92.23703,1,0-20%
4,14000US37001020501,205.01,Alamance,92.7,35.1,23.3,76.7,15.6,76.1,13.3,⋯,18.9,0,0.0,53.1,25.7,10.0,278233.8,92.23703,1,0-20%
5,14000US37001020502,205.02,Alamance,99.0,32.1,30.0,70.0,8.9,40.7,35.7,⋯,27.8,100,14.2,19.5,14.7,25.0,278233.8,92.23703,1,0-20%
6,14000US37001020601,206.01,Alamance,97.3,51.7,17.5,82.5,27.8,96.9,0.9,⋯,0.0,0,100.0,0.0,6.7,3.6,278233.8,92.23703,1,0-20%


# Variable Filtering
RF imputation needs at least 25% of data in order to impute the missing data, therefore variables with less than 50% won't be imputed. This filtering will be done within each county.

In [33]:
# only keeping variables with at least 25% of data in each county
variable_presence_df = acs_whp_df[,1:31] %>% 
    pivot_longer(cols = 4:31, names_to = "Variable", values_to = "Value") %>%
    # creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(Value), 0, 1), all_count = 1) %>%
    group_by(County, Variable) %>%
    summarize(Variable_Presence = sum(data_count)/ sum(all_count) * 100) %>%
    filter(Variable_Presence > 25) %>%
    arrange(-Variable_Presence)
    #select(-Census_Tract) %>%
    #unique()

head(variable_presence_df)

`summarise()` has grouped output by 'County'. You can override using the
`.groups` argument.


County,Variable,Variable_Presence
<chr>,<chr>,<dbl>
Alamance,Avg_ACRES,100
Alamance,English_Speakers_Only,100
Alamance,Hispanic_Latino,100
Alamance,Median_Age,100
Alamance,Non_Proficient_English_Speakers,100
Alamance,Population_Over_18,100


# Random Forest Imputation
Generating missing values using random forest (RF).

In [42]:
b = variable_presence_df %>% filter (County == "Ashe")
keeping = c(colnames(acs_whp_df)[1:3], b$Variable)

acs_whp_df[,keeping]

GEO_ID,Census_Tract,County,Avg_ACRES,English_Speakers_Only,Hispanic_Latino,Median_Age,Non_Proficient_English_Speakers,Population_Over_18,Population_Over_65,⋯,Race_Black,Race_Indigenous,Race_Mixed,Race_Other,Race_White,Sex_Ratio,Poverty_Black,Proficient_in_Non_English_Language,Poverty_Asian,Poverty_Other_Race
<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14000US37001020100,201.00,Alamance,278233.8,87.6,9.5,37.3,3.9,78.1,15.1,⋯,28.5,0.3,1.9,6.5,61.0,87.2,54.8,31.2,0.0,45.3
14000US37001020200,202.00,Alamance,278233.8,68.3,28.4,31.0,16.0,70.9,11.2,⋯,29.2,0.9,1.8,11.7,51.9,78.3,44.6,50.4,0.0,0.0
14000US37001020400,204.00,Alamance,278233.8,80.8,21.0,30.5,10.1,68.6,10.1,⋯,59.0,0.2,1.4,19.9,19.6,78.1,19.9,52.6,NA,69.0
14000US37001020501,205.01,Alamance,278233.8,86.9,14.5,35.1,9.9,76.7,15.6,⋯,13.3,0.0,0.8,8.3,76.1,92.7,18.9,75.8,0.0,0.0
14000US37001020502,205.02,Alamance,278233.8,75.6,23.5,32.1,13.2,70.0,8.9,⋯,35.7,0.0,6.2,15.1,40.7,99.0,27.8,54.3,100.0,14.2
14000US37001020601,206.01,Alamance,278233.8,93.7,3.6,51.7,0.8,82.5,27.8,⋯,0.9,0.2,1.3,0.2,96.9,97.3,0.0,12.4,0.0,100.0
14000US37001020602,206.02,Alamance,278233.8,97.6,6.5,49.2,0.9,83.2,30.8,⋯,3.7,0.0,0.6,0.5,94.3,93.3,0.0,38.7,0.0,0.0
14000US37001020701,207.01,Alamance,278233.8,90.0,9.1,47.2,2.2,83.5,22.0,⋯,11.8,0.8,3.7,0.6,82.7,99.7,16.6,22.0,0.0,0.0
14000US37001020702,207.02,Alamance,278233.8,87.7,17.9,36.9,5.4,78.6,16.5,⋯,31.3,0.0,0.5,10.9,57.3,114.4,52.2,43.9,NA,32.2


In [47]:
a = acs_whp_df[,keeping] %>% 
 filter(County == "Ashe")

# test
#a[a == NA] = " "
matrix_a = as.matrix(a[,4:length(keeping)])
matrix_a

missForest(matrix_a)

Avg_ACRES,English_Speakers_Only,Hispanic_Latino,Median_Age,Non_Proficient_English_Speakers,Population_Over_18,Population_Over_65,Population_Under_18,Poverty_Adult,Poverty_Children,⋯,Race_Black,Race_Indigenous,Race_Mixed,Race_Other,Race_White,Sex_Ratio,Poverty_Black,Proficient_in_Non_English_Language,Poverty_Asian,Poverty_Other_Race
274732.1,93.9,6.3,45.1,0.6,83.4,21.8,16.6,18.1,25.8,⋯,0.7,0.2,2.4,0.0,96.7,109.3,0,9.1,NA,NA
237671.3,100.0,0.5,51.2,0.0,82.1,26.1,17.9,24.9,54.8,⋯,0.0,0.7,1.0,0.0,98.3,106.0,NA,NA,NA,NA
274732.1,92.6,6.8,46.3,1.4,78.1,21.5,21.9,15.7,20.0,⋯,0.9,0.0,2.3,0.8,94.8,89.5,80,19.6,3.9,16.7


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”


ERROR: Error in if (n == 0) stop("data (x) has 0 rows"): argument is of length zero


In [41]:
# imputing sociodemographic data using RF
random_forest_imputation = function(preimputed_df, presence_df){
    
    imputed_df = data.frame()
    # iterating through each df grouped by county 
    for (i in 1:length(unique(preimputed_df$County))){ 
        
        # variable presence df first
        county_variable_presence_df = presence_df %>%
            filter(County == unique(preimputed_df$County)[i])
        variables_to_keep = c(colnames(preimputed_df)[1:3], county_variable_presence_df$Variable)
        
        # then the preimputed df
        county_preimputed_df = preimputed_df[,variables_to_keep] %>%
            filter(County == unique(preimputed_df$County)[i])
        print(unique(preimputed_df$County)[i])
        
        imputed_RF_object = missForest(as.matrix(county_preimputed_df[,4:length(variables_to_keep)]))
        imputed_RF_df = imputed_RF_object$ximp
        #imputed_values_df = data.frame(cbind(county_preimputed_df[,c(1:3,33,34)], imputed_RF_df)) 

        # saving imputed values
        #imputed_df = rbind(imputed_df, imputed_values_df)

    }
    
    #return(imputed_final_df)
}

# calling function
imputed_df = random_forest_imputation(acs_whp_df, variable_presence_df)
head(imputed_df)

[1] "Alamance"
[1] "Alexander"


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response h

[1] "Alleghany"


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”


[1] "Anson"


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response h

[1] "Ashe"


Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”
Warning message in randomForest.default(x = obsX, y = obsY, ntree = ntree, mtry = mtry, :
“The response has five or fewer unique values.  Are you sure you want to do regression?”


ERROR: Error in if (n == 0) stop("data (x) has 0 rows"): argument is of length zero


In [9]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_ACS_WHP_Data.csv"), row.names = FALSE)